In [12]:
import pandas as pd
import numpy as np
import ast
import re
import chardet
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration

%run "./support_functions.ipynb"

file_path = "./BPMN.csv"
output_file_path = "./BPMNcleaned.csv"
removed_rows_file_path = "./BPMN_removed_short_rows.csv"

# vedo l'encoding del file
input_file_encoding = get_file_encoding(file_path)
print(f"L'encoding del file '{file_path}' è: {input_file_encoding}")

# Save the data of the .csv file in a Variable
df = pd.read_csv(file_path, sep=';', engine='python', encoding=input_file_encoding)

# Prendo solo le utlime due colonne
df = df.iloc[:, [0,-2]]

# Elimino i duplicati
df = df.drop_duplicates()

# Pulisco le colonne dai "^^^" se x è una stringa tolto ^^^ altrimenti ritorno x
df['Labels'] = df['Labels'].fillna('').apply(lambda x: 
                                             x.split("^^^") if isinstance(x, str) else x)

# Salvo le righe con meno di 4 parole oppure con solo lettere singole o doppie in una struttura
removed_rows_df = df[df['Labels'].apply(lambda x: 
                                        sum(1 for item in x if isinstance(item, str) and len(item) <= 2) > len(x) / 2 or len(x) < 4)]

# Salvo le righe eliminate in un .csv
removed_rows_df.to_csv(removed_rows_file_path, index=False, sep=';')

# Mi porto nel databse 
df = df[~df['Labels'].apply(lambda x: 
                            sum(1 for item in x if isinstance(item, str) and len(item) <= 2) > len(x) / 2 or len(x) < 4)]



# Save the file .csv
df.to_csv(output_file_path, index=False, sep=';')

df.info()
df.head(15)

# Print removed rows
print("Righe rimosse:")
print(removed_rows_df)

#print(type(df))
#print(type(df['Labels']))
#print(type(df['Labels'].iloc[2]))
#print(type(df['Labels'].iloc[2][2]))



L'encoding del file './BPMN.csv' è: utf-8
<class 'pandas.core.frame.DataFrame'>
Index: 13348 entries, 0 to 22575
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   CollectionName  13348 non-null  object
 1   Labels          13348 non-null  object
dtypes: object(2)
memory usage: 312.8+ KB
Righe rimosse:
      CollectionName                                             Labels
6                BIT                             [s00000520, s00000527]
10               BIT                  [s00000642, s00000643, s00000645]
11               BIT                                        [s00000026]
12               BIT                             [s00000527, s00000671]
16               BIT                                        [s00000716]
...              ...                                                ...
18836          BPMAI  [A, AC, AC, AG, AG, AJ, AJ, AK, AK, AM, AM, AP...
19427        Camunda  [Vorgangschliessen, Zahl

In [1]:
import ast
import chardet
import os
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader

%run "./support_functions.ipynb"

input_file_path = './BPMNcleaned.csv'
output_file_path = './BPMNcleanedlanguages.csv'

input_file_encoding = get_file_encoding(input_file_path)
print(f"L'encoding del file '{input_file_path}' è: {input_file_encoding}.")

df = pd.read_csv(input_file_path, sep=';', engine='python', encoding=input_file_encoding)

# Creazione del dataset con i suoi metodi richiesti
class CustomDataset(Dataset):
    def __init__(self, path):
        
         # Prima di tutto controlliamo che il path esista
        if not os.path.isfile(path):
            # Se il path non è valido allora solleviamo un'eccezione
            raise ValueError('Invalid `path` variable! Needs to be a file')
        
        # Carico il file .csv
        self.df = pd.read_csv(path, sep=';', engine='python', encoding=get_file_encoding(path))

        self.data = df['Labels']
        self.length = len(self.data)
        
    def __len__(self):
        return self.length
    
    def __getitem__(self, index):
        sample = self.data[index]
        return sample
# Creazione del dataloader che sarà necessario ai fini di usare 'T5'
def create_dataloader(dataframe, batch_size, shuffle=True):
    dataset = CustomDataset(dataframe)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)
    return dataloader

L'encoding del file './BPMNcleaned.csv' è: utf-8.


In [2]:
import ast
import chardet
import os
import torch
import torch.nn as nn
import concurrent.futures
import pandas as pd
from ast import literal_eval
from langdetect import detect
from tqdm import tqdm
from transformers import T5Tokenizer, T5ForConditionalGeneration
from torch.utils.data import Dataset, DataLoader, TensorDataset


%run "./support_functions.ipynb"

input_file_path = './BPMNcleaned.csv'
output_file_path = './BPMNcleanedlanguages.csv'

batch_size = 32

tokenizer = T5Tokenizer.from_pretrained("t5-small",legacy=False)
model = T5ForConditionalGeneration.from_pretrained("t5-small")
tqdm.pandas()


# Vedo l'encoding del file
input_file_encoding = get_file_encoding(input_file_path)
print(f"L'encoding del file '{input_file_path}' è: {input_file_encoding}.")

# Leggo il file CSV in un DataFrame
df = pd.read_csv(input_file_path, sep=';', engine='python', encoding=input_file_encoding)

# Funzione per la pulizia delle etichette con T5 utilizzando il batch processing
def clean_labels_with_t5_batch(labels_batch):
    input_ids = tokenizer.batch_encode_plus(labels_batch, return_tensors="pt", max_length=512, truncation=True, padding=True)['input_ids']
    output_ids = model.generate(input_ids=input_ids, max_length=512)
    cleaned_labels_batch = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
    return cleaned_labels_batch


# Imposto la lingua che vogliamo mantenere e ritorno se la stringa appartiene 
def is_desired_language(text):
    try:
        return detect(text)=='en'
    except:
        return False
    
# Metodo che cicla ogni etichetta
def is_desired_language_list(labels):
    return any(is_desired_language(str(label)) for label in labels)

# Filtro ogni riga del dataframe e successivamente pulisco le righe non consone
def filter_dataframe(df):
    df['Labels'] = df['Labels'].apply(lambda labels: literal_eval(labels))
    df['is_desired_language'] = df['Labels'].apply(is_desired_language_list)
        
    discarded_rows = df[~df['is_desired_language']]
    filtered_df = df[df['is_desired_language']].drop(['is_desired_language'], axis=1)
    
    # stampo le righe eliminate
    print(f"Sono state eliminate {len(discarded_rows)} righe.")
    print(f"Righe eliminate: \n {discarded_rows}")
    #print(discarded_rows)

    return filtered_df

# Creazione del dataloader che creerà un dataset utile da passare al 'T5' 
dataloader = create_dataloader(input_file_path, batch_size)
# Itero ogni batch del dataloader e applichiamo il 'T5'
for batch in dataloader:
    clean_labels_with_t5_batch(batch)

# Applico il filtro al DataFrame
filtered_df = filter_dataframe(df)

# Salvo il DataFrame risultante in un nuovo file CSV
filtered_df.to_csv(output_file_path, index=False, sep=';')


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


L'encoding del file './BPMNcleaned.csv' è: utf-8.
Sono state eliminate 5370 righe.
Righe eliminate: 
       CollectionName                                             Labels  \
675            BPMAI  [ArztuntersuchtnachVollst?хndigkeit, Gewebewir...   
678            BPMAI  [ANO, ANO, DefaultInputSet, DefaultInputSet, D...   
679            BPMAI  [Datenbereitsvorhanden?, Ja, Ja, Ja, Kontendat...   
687            BPMAI     [subprocessname, subprocessname, task1, task2]   
691            BPMAI  [Auftragsbest?хtigungerstellen, Bestelldatenau...   
...              ...                                                ...   
13341            eCH  [52PatentpflichtAnlassabkl?хren, Bewilligunger...   
13342            eCH  [52PatentpflichtAnlassabkl?хren, AusnahmenzurA...   
13344            eCH  [Antragauf?ҐnderungVernehmlassungsweglangstatt...   
13345            eCH  [Abteilungsleiter, allesi.O., allesi.O., Anpas...   
13347            eCH  [BesprechungmitBeteiligten, Bewilligungerteile...  

In [ ]:
import ast
import chardet
import pandas as pd

%run "./support_functions.ipynb"

input_file_path = './BPMNcleanedlanguages.csv'
output_training_file = './BPMNtraining.csv'
output_testing_file = './BPMNtesting.csv'

# vedo l'encoding del file
input_file_encoding = get_file_encoding(input_file_path)
print(f"L'encoding del file '{input_file_path}' è: {input_file_encoding}")

# Loading the DataFrame
df = pd.read_csv(input_file_path, sep=';', engine='python', encoding=input_file_encoding)

# Funzione che divide il DataFrame in proporzion
def split_train_test(data, test_ratio):
    shuffled_indices = np.random.permutation(len(data))
    test_set_size = int(len(data) * test_ratio)
    test_indices = shuffled_indices[:test_set_size]
    train_indices = shuffled_indices[test_set_size:]
    return data.iloc[train_indices], data.iloc[test_indices]

# Funzione che da un dataframe ritorna un dataframe di dataframe divisi
def split_by_collectionname(data):
    divided_data = {}
    for collectionname in data['CollectionName'].unique():
        divided_data[collectionname] = data[data['CollectionName'] == collectionname].copy()
    return divided_data

# Funzione che per ogni chiave (collectionname) divide in train e testing
def creating_train_test_dataframe(data, test_ratio):
    training_df = []
    testing_df = []
    # Dividi il DataFrame per CollectionName
    divided_data = split_by_collectionname(data)
    # Per ogni CollectionName, dividi il DataFrame in train e test
    for collectionname, data in divided_data.items():
        train_df, test_df = split_train_test(data, test_ratio)
        training_df.append(train_df)
        testing_df.append(test_df)
        
        training_data=pd.concat(training_df)
        testing_data=pd.concat(testing_df) 
    
    return training_data, testing_data  
        
# Prende randomicamente e in percentuale le righe del df per training e testing
# training_data, testing_data = split_train_test(df, 0.3)
training_data, testing_data = creating_train_test_dataframe(df, 0.3)


# Stampo le prime 20 righe di training e testing
# Stampa il numero dei diversi domini nel DataFrame di testing
print("Numero di diversi domini nel testing_data:")
print(testing_data['CollectionName'].value_counts())
print(testing_data.head())

# Stampa il numero dei diversi domini nel DataFrame di training
print("\nNumero di diversi domini nel training_data:")
print(training_data['CollectionName'].value_counts())
print(training_data.head())

# salva in file rispettivi per training e testing
training_data.to_csv(output_training_file, index=False, sep=';')
testing_data.to_csv(output_testing_file, index=False, sep=';')